In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
from xgboost import XGBClassifier

In [3]:
data = pd.read_csv('cleaned_data.csv', index_col= 0)
data = data.sample(frac = 1)
from sklearn.model_selection import train_test_split

In [4]:
X = data.drop('Label', axis =1)
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)


In [5]:
learning_rate_range = np.arange(0.01, 1, 0.05)
test_XG = [] 
train_XG = []
for lr in learning_rate_range:
    xgb_classifier = xgb.XGBClassifier(eta = lr)
    xgb_classifier.fit(X_train, y_train)
    train_XG.append(xgb_classifier.score(X_train, y_train))
    test_XG.append(xgb_classifier.score(X_test, y_test))

In [6]:
test_XG

[0.913656161926723,
 0.9605431719190366,
 0.9792467332820907,
 0.986420702024084,
 0.9894952600563669,
 0.9907763259031515,
 0.9884704073789393,
 0.9902638995644376,
 0.9884704073789393,
 0.9884704073789393,
 0.9882141942095823,
 0.9897514732257238,
 0.9907763259031515,
 0.9900076863950807,
 0.9894952600563669,
 0.98923904688701,
 0.9887266205482962,
 0.98923904688701,
 0.9887266205482962,
 0.9869331283627979]

In [7]:
train_XG

[0.9203821656050956,
 0.9747419283988579,
 0.991544036898748,
 0.9980232813529542,
 0.9994509114869317,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [8]:
np.arange(0.1, 1, 0.05)

array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ,
       0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [9]:
model = xgb_classifier = xgb.XGBClassifier(eta = 0.3)

In [10]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [11]:
test_output = model.predict(X_test)

In [12]:
train_output = model.predict(X_train)

In [13]:
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score

score_list = [f1_score , precision_score , recall_score , accuracy_score]
def eval_model(score_list, y_true, y_pred):
    results = []
    for score in score_list:
        value = score(y_true, y_pred)
        results.append({"score_name" : score, "value": value})
    return pd.DataFrame(results)

In [14]:
#train performance 

eval_model(score_list, y_test, test_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.988482
1,<function precision_score at 0x12fa50ee0>,0.981199
2,<function recall_score at 0x12fa50f70>,0.995874
3,<function accuracy_score at 0x12fa50790>,0.988470


In [15]:
eval_model(score_list, y_train, train_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,1.0
1,<function precision_score at 0x12fa50ee0>,1.0
2,<function recall_score at 0x12fa50f70>,1.0
3,<function accuracy_score at 0x12fa50790>,1.0


In [16]:
test_data = pd.read_csv('hack4ghana/Test.csv', index_col=0)
test = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [17]:
test['Target'] = model.predict(test)

In [18]:
submission3 = test[['Target']]

In [19]:
submission3['Target'].value_counts()

Target
0    2791
1    1027
Name: count, dtype: int64

In [20]:
submission3.to_csv('third_submission.csv')

In [25]:
from sklearn.model_selection import RandomizedSearchCV

In [30]:
params = {
    'booster': ['gbtree', 'gblinear', 'dart'],
    'max_depth': np.arange(6,15, 1),
    'min_child_weight': np.arange(0, 10),
    'eta' : [0.35],
    'sampling':['uniform', 'subsample', 'gradient_based'],
    'alpha' : [0, 0.1, 0.5, 1],
    'tree_method': ['auto', 'exact','approx', 'hist', ]
}

In [32]:
grid_search = RandomizedSearchCV(boost, param_distributions= params, cv =5, n_iter= 1)

In [33]:
grid_search.fit(X_train, y_train)

[12:58:20] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[12:58:24] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[12:58:28] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[12:58:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[12:58:36] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[12:58:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-ar

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=1,
                   param_distributions={'alpha': [0, 0.1, 0.5, 1],
                                        'booster': ['gbtree', 'gblinear',
                                                    'dart'],
                                        'eta': [0.35],
                                        'max_depth': array([ 6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_child_weight': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'sampling': ['uniform', 'subsample',
                                                     'gradient_based'],
                                        'tree_method': ['auto', 'exact',
                                                        'approx', 'hist']})

In [34]:
grid_search.best_params_

{'tree_method': 'hist',
 'sampling': 'gradient_based',
 'min_child_weight': 8,
 'max_depth': 8,
 'eta': 0.35,
 'booster': 'dart',
 'alpha': 1}

In [35]:
second_model = XGBClassifier(tree_method = 'hist', sampling = 'gradient_based', min_child_weight= 8,max_depth = 8, eta = 0.35, booster ='dart', alpha = 1)

In [36]:
second_model.fit(X_train, y_train)

[13:02:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.



XGBClassifier(alpha=1, base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.35, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=8, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [39]:
second_test_output = second_model.predict(X_test)

In [38]:
second_train_output = second_model.predict(X_train)

In [40]:
eval_model(score_list, y_test,second_test_output )

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.987718
1,<function precision_score at 0x12fa50ee0>,0.980193
2,<function recall_score at 0x12fa50f70>,0.995358
3,<function accuracy_score at 0x12fa50790>,0.987702


In [41]:
eval_model(score_list, y_train, second_train_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.999124
1,<function precision_score at 0x12fa50ee0>,0.999562
2,<function recall_score at 0x12fa50f70>,0.998686
3,<function accuracy_score at 0x12fa50790>,0.999121


In [42]:
second_test = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [43]:
second_test['Target'] = second_model.predict(second_test)

In [44]:
second_test['Target'].value_counts()

Target
0    2787
1    1031
Name: count, dtype: int64

In [45]:
submission3 = second_test[['Target']]

In [47]:
submission3.to_csv('fourth_submission.csv')

In [48]:
params_two= {
    'eta': np.arange(0.01, 1, 0.05),
    'booster': ['gbtree', 'gblinear', 'dart'],
    'max_depth': np.arange(6,15, 1),
    'min_child_weight': np.arange(0, 10),
    'sampling':['uniform', 'subsample', 'gradient_based'],
    'alpha' : [0, 0.1, 0.5, 1],
    'tree_method': ['auto', 'exact','approx', 'hist']
}

In [51]:
random_search = RandomizedSearchCV(boost, param_distributions= params_two, cv =5, n_iter= 1)

In [52]:
random_search.fit(X_train, y_train)

[13:13:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[13:13:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[13:13:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[13:13:11] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[13:13:11] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[13:13:11] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-ar

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'alpha': [0, 0.1, 0.5, 1],
                                        'booster': ['gbtree', 'gblinear',
                                                    'dart'],
                                        'eta': array([0.01, 0.06, 0.11, 0.16, 0.21, 0.26, 0.31, 0.36, 0.41, 0.46, 0.51,
       0.56, 0.61, 0.66, 0.71, 0.76, 0.81, 0.86, 0.91, 0.96]),
                                        'max_depth': array([ 6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_child_weight': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'sampling': ['uniform', 'subsample',
                                                     'gradient_based'],
                                        'tree_method': ['auto', 'exact',
                                                        'approx', 'hist']})

In [53]:
random_search.best_params_

{'tree_method': 'hist',
 'sampling': 'gradient_based',
 'min_child_weight': 7,
 'max_depth': 13,
 'eta': 0.66,
 'booster': 'gbtree',
 'alpha': 0}

In [54]:
third_model = XGBClassifier(tree_method = 'hist', sampling = 'gradient_based', min_child_weight= 8,max_depth = 8, eta = 0.66, booster ='dart', alpha = 1)

In [56]:
third_model.fit(X_train, y_train)

[13:14:46] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.



XGBClassifier(alpha=1, base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.66, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=8, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [57]:
third_test_output = third_model.predict(X_test)

In [58]:
third_train_output = third_model.predict(X_train)

In [59]:
eval_model(score_list, y_test, third_test_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.989759
1,<function precision_score at 0x12fa50ee0>,0.982715
2,<function recall_score at 0x12fa50f70>,0.996906
3,<function accuracy_score at 0x12fa50790>,0.989751


In [60]:
eval_model(score_list, y_train, third_train_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.999562
1,<function precision_score at 0x12fa50ee0>,0.999562
2,<function recall_score at 0x12fa50f70>,0.999562
3,<function accuracy_score at 0x12fa50790>,0.999561


In [55]:
third_test = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [61]:
third_test['Target'] = third_model.predict(third_test)

In [62]:
submission4 = third_test[['Target']]

In [64]:
submission4['Target'].value_counts()

Target
0    2787
1    1031
Name: count, dtype: int64

In [65]:
submission4.to_csv('fifth_submission.csv')

In [70]:
import catboost
from catboost import CatBoostClassifier

model_params = {
    'iterations': 10, 
    'loss_function': 'Logloss',
    'train_dir': 'crossentropy',
    'allow_writing_files': False,
    'random_seed': 42,
}

fourth_model = CatBoostClassifier(**model_params)
fourth_model.fit(X_train, y_train, verbose=True)

Learning rate set to 0.5
0:	learn: 0.4839726	total: 8.4ms	remaining: 75.6ms
1:	learn: 0.3941571	total: 10.7ms	remaining: 42.8ms
2:	learn: 0.3337508	total: 13ms	remaining: 30.2ms
3:	learn: 0.2975641	total: 25.2ms	remaining: 37.8ms
4:	learn: 0.2769108	total: 48.1ms	remaining: 48.1ms
5:	learn: 0.2609278	total: 50.3ms	remaining: 33.6ms
6:	learn: 0.2459626	total: 52.7ms	remaining: 22.6ms
7:	learn: 0.2310626	total: 55.4ms	remaining: 13.8ms
8:	learn: 0.2202941	total: 58.5ms	remaining: 6.5ms
9:	learn: 0.2090221	total: 64.8ms	remaining: 0us


In [74]:
fourth_test_output = fourth_model.predict(X_test)

In [75]:
fourth_train_output =  fourth_model.predict(X_train)

In [76]:
eval_model(score_list, y_test, fourth_test_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.917671
1,<function precision_score at 0x12fa50ee0>,0.893888
2,<function recall_score at 0x12fa50f70>,0.942754
3,<function accuracy_score at 0x12fa50790>,0.915962


In [77]:
param_v3 = {
    'eta': np.arange(0.01, 1, 0.05)
}

In [78]:
random_search = RandomizedSearchCV(boost, param_distributions= param_v3, cv =5, n_iter= 1)

In [79]:
random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_cat_to_onehot=None,
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=1,
                   param_distributions={'eta': array([0.01, 0.06, 0.11, 0.16, 0.21, 0.26, 0.31, 0.36, 0.41, 0.46, 0.51,
       0.56, 0.61, 0.66, 0.71, 0.76, 0.81, 0.86, 0.91, 0.96])})

In [80]:
random_search.best_params_

{'eta': 0.91}

In [81]:
model5 = XGBClassifier( eta = 0.91)

In [82]:
model5.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.91, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [83]:
model5_test_output = model5.predict(X_test)

In [84]:
model5_train_output = model5.predict(X_train)

In [85]:
eval_model(score_list,y_test, model5_test_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.988741
1,<function precision_score at 0x12fa50ee0>,0.981209
2,<function recall_score at 0x12fa50f70>,0.996390
3,<function accuracy_score at 0x12fa50790>,0.988727


In [86]:
eval_model(score_list,y_train, model5_train_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,1.0
1,<function precision_score at 0x12fa50ee0>,1.0
2,<function recall_score at 0x12fa50f70>,1.0
3,<function accuracy_score at 0x12fa50790>,1.0


In [88]:
test5 = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [89]:
test5['Target'] = model5.predict(test5)

In [90]:
submission7 = test5[['Target']]

In [92]:
submission7['Target'].value_counts()


Target
0    2797
1    1021
Name: count, dtype: int64

In [93]:
submission7.to_csv('seventh_submission.csv')

In [114]:
grid_search_v2 = RandomizedSearchCV(boost, param_distributions= param_v4, cv =5, n_iter= 1, random_state= 2,)

In [115]:
grid_search_v2.fit(X_train, y_train)

[15:06:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[15:07:00] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[15:07:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[15:07:01] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[15:07:02] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.

[15:07:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-ar

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           predictor=None, random_state=None, ...),
                   n_iter=1,
                   param_distributions={'alpha': [0, 0.1, 0.5, 1],
                                        'booster': ['gbtree', 'gblinear',
                                                    'dart'],
                                        'eta': [0.35],
                                        'max_depth': array([ 6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'min_child_weight': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'objective': ['binary:logistic'],
                                        'sampling': ['uniform', 'subsample',
                                                     'gradient_based'],
                                        'tree_method': ['auto', 'exact',
                                                        'approx', 'hist']},
                   random_state=2)

In [116]:
grid_search_v2.best_params_

{'tree_method': 'auto',
 'sampling': 'subsample',
 'objective': 'binary:logistic',
 'min_child_weight': 1,
 'max_depth': 13,
 'eta': 0.35,
 'booster': 'gbtree',
 'alpha': 0.5}

In [118]:
last_model =XGBClassifier(tree_method='auto',sampling='subsample',objective= 'binary:logistic',min_child_weight=1, max_depth= 13, eta = 0.3, booster='gbtree',alpha=0.5)



In [119]:
last_model.fit(X_train,y_train)



[15:10:26] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "sampling" } are not used.



XGBClassifier(alpha=0.5, base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=13,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [120]:
last_model_test_output = last_model.predict(X_test)

In [121]:
last_model_train_output= last_model.predict(X_train)

In [122]:
eval_model(score_list,y_train, last_model_train_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,1.0
1,<function precision_score at 0x12fa50ee0>,1.0
2,<function recall_score at 0x12fa50f70>,1.0
3,<function accuracy_score at 0x12fa50790>,1.0


In [123]:
eval_model(score_list,y_test, last_model_test_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.988747
1,<function precision_score at 0x12fa50ee0>,0.980720
2,<function recall_score at 0x12fa50f70>,0.996906
3,<function accuracy_score at 0x12fa50790>,0.988727


In [107]:
test_l = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)

In [108]:
test_l['Target'] = last_model.predict(test_l)

In [109]:
submission_l = test_l[['Target']]

In [111]:
submission_l['Target'].value_counts()

Target
0    2804
1    1014
Name: count, dtype: int64

In [112]:
submission_l.to_csv('submission9.csv')

In [124]:
param_v4 = {
    'booster': ['gbtree', 'gblinear', 'dart'],
    'max_depth': np.arange(6,15, 1),
    'min_child_weight': np.arange(0, 10),
    'eta' : [0.25],
    'sampling':['uniform', 'subsample', 'gradient_based'],
    'alpha' : [0, 0.1, 0.5, 1],
    'tree_method': ['auto', 'exact','approx', 'hist', ],
    'objective': ['binary:logistic']
}

In [ ]:
last_model.early_stopping_rounds()

In [176]:
model_q = XGBClassifier(eta = 0.25)


In [177]:
model_q.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.25, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [178]:
test_q_output = model_q.predict(X_test)

In [179]:
eval_model(score_list, y_test, test_q_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.990256
1,<function precision_score at 0x12fa50ee0>,0.984702
2,<function recall_score at 0x12fa50f70>,0.995874
3,<function accuracy_score at 0x12fa50790>,0.990264


In [180]:
train_q_output = model_q.predict(X_train)

In [181]:
eval_model(score_list,y_train, train_q_output)

,score_name,value
0,<function f1_score at 0x12fa50af0>,0.999890
1,<function precision_score at 0x12fa50ee0>,1.000000
2,<function recall_score at 0x12fa50f70>,0.999781
3,<function accuracy_score at 0x12fa50790>,0.999890


In [182]:
test_l = test_data.drop(['Disc Writing/sec', 'Bytes Sent/sent', 'Pages Read/sec'], axis=1)
test_l['Target'] = last_model.predict(test_l)
submission_l = test_l[['Target']]
submission_l['Target'].value_counts()
submission_l.to_csv('submission10.csv')